# 원, 삼각형, 사각형 구분하기
김태영의 컨볼루션신경망모델 만들어보기 [참조]
https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/
## 3. Data Augmentation을 사용한 CNN 모델

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
!unzip -qq handwriting_shape.zip -d handwriting_shape

In [3]:
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

### 데이터부풀리기(Augmentation)

In [6]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [7]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15, #15도각도로회전
                                   width_shift_range=0.1,#10px이내 좌우이동
                                   height_shift_range=0.1,#10px이내 수직이동
                                   shear_range=0.5, #시계반대방향으로 밀림 강도를 라디안으로
                                   zoom_range=[0.8, 2.0], #확대/축소
                                   horizontal_flip=True,#수평방향으로 뒤집기
                                   vertical_flip=True, #수직방향으로 뒤집기
                                   fill_mode='nearest')

In [9]:
!mkdir preview

In [10]:
#어떻게 동작하는지 샘플 먼저보기
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!! train_datagen.flow핵심ㅋ.ㅋ 
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', 
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

In [11]:
!ls preview

tri_0_2037.png	tri_0_3597.png	tri_0_5139.png	tri_0_7189.png	tri_0_9398.png
tri_0_2546.png	tri_0_3662.png	tri_0_5553.png	tri_0_7405.png	tri_0_9855.png
tri_0_2731.png	tri_0_4213.png	tri_0_6293.png	tri_0_7642.png	tri_0_9864.png
tri_0_3313.png	tri_0_4345.png	tri_0_6390.png	tri_0_8188.png
tri_0_3339.png	tri_0_4401.png	tri_0_654.png	tri_0_872.png
tri_0_3448.png	tri_0_4880.png	tri_0_6921.png	tri_0_8816.png
tri_0_3526.png	tri_0_4950.png	tri_0_7142.png	tri_0_9307.png


### 데이터셋 생성하기

In [12]:
#여기바뀜
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.


In [13]:
!ls hard_handwriting_shape/train
#파일3가지를 만들어진것 확인
#circle	rectangle  triangle

circle	rectangle  triangle


In [14]:
#테스트셋은 그대로~
test_datagen = ImageDataGenerator(rescale=1./255) #0~1사이의값으로 정규화
#테스트셋을 넘파이 어레이로 만들기
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [15]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [16]:
train_generator.filenames[0]

'circle/circle001.png'

### 모델 정의/설정/학습

In [17]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

#### 모델 학습시키기
-첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.  
-steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.  
-epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.  
-validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 test_generator으로 지정.  
-validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [19]:
model.fit_generator( # 기존에알고있던것에서 _generator추가됨
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
15/15 [==============================] - 1s 38ms/step - loss: 1.4168 - accuracy: 0.3080 - val_loss: 1.0972 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 24ms/step - loss: 1.0647 - accuracy: 0.3902 - val_loss: 1.0771 - val_accuracy: 0.6000
Epoch 3/200
15/15 [==============================] - 0s 23ms/step - loss: 0.9653 - accuracy: 0.6317 - val_loss: 1.0479 - val_accuracy: 0.4000
Epoch 4/200
15/15 [==============================] - 0s 24ms/step - loss: 0.7179 - accuracy: 0.7350 - val_loss: 1.2654 - val_accuracy: 0.6000
Epoch 5/200
15/15 [==============================] - 0s 24ms/step - loss: 0.5731 - accuracy: 0.7287 - val_loss: 1.4148 - val_accuracy: 0.6667
Epoch 6/200
15/15 [==============================] - 0s 24ms/step - loss: 0.6893 - accuracy: 0.6789 - val_loss: 1.3834 - val_accuracy: 0.4000
Epoch 7/200
15/15 [==============================] - 0s 25ms/step - loss: 0.5360 - accuracy: 0.7868 - val_loss: 1.3976 - val_accuracy: 0.6000
Epoch 

### 모델평가하기

In [20]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5) #추가됨 _generator
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
#66.67% 

-- Evaluate --
accuracy: 66.67%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
